In [1]:

!pip install implicit

     ---------------------------------------- 0.0/647.0 kB ? eta -:--:--
     ---- ---------------------------------- 81.9/647.0 kB 1.5 MB/s eta 0:00:01
     --------------- ---------------------- 256.0/647.0 kB 2.3 MB/s eta 0:00:01
     ---------------------------------- --- 583.7/647.0 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 647.0/647.0 kB 3.1 MB/s eta 0:00:00


In [117]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.evaluation import train_test_split, ranking_metrics_at_k
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import train_test_split
import json


In [67]:
BOTIFY_DATA_DIR = "../botify/data/"
data = pd.read_json("../data_large.json", lines=True)[["user", "time", "track"]].copy()
data.head(3)

,user,time,track
0,3255,1.00,2659
1,3255,0.40,5510
2,3255,0.04,7417


In [91]:
data['user'].max()

9999

In [68]:
data["normalized_time"] = data.groupby("user")["time"].transform(lambda time: time - time.mean())
data.head(3)

,user,time,track,normalized_time
0,3255,1.00,2659,0.693333
1,3255,0.40,5510,0.093333
2,3255,0.04,7417,-0.266667


In [128]:
users, user_inds = np.unique(list(data["user"]), return_inverse=True)
items, item_inds = np.unique(list(data["track"]), return_inverse=True)

# item_user = csr_matrix((np.ones(data.shape[0]), (item_inds, user_inds)), shape=(len(items), len(users)))
item_user = csr_matrix((np.ones(data.shape[0]), (user_inds, item_inds)), shape=(len(users), len(items)))
item_user

<9996x49947 sparse matrix of type '<class 'numpy.float64'>'
	with 545813 stored elements in Compressed Sparse Row format>

In [129]:
# random split
train, test = train_test_split(item_user)
print('train', repr(train), 'test', repr(test), sep='\n')

train
<9996x49947 sparse matrix of type '<class 'numpy.float64'>'
	with 436641 stored elements in Compressed Sparse Row format>
test
<9996x49947 sparse matrix of type '<class 'numpy.float64'>'
	with 109172 stored elements in Compressed Sparse Row format>


In [130]:
model = AlternatingLeastSquares(factors=50, regularization=0.05, iterations=15)
model.fit(train)

  0%|          | 0/15 [00:00<?, ?it/s]

In [100]:
train

<49947x9996 sparse matrix of type '<class 'numpy.float64'>'
	with 436452 stored elements in Compressed Sparse Row format>

In [98]:
liked = train.T[534].nonzero()[1]
liked

array([  422,   937,  1036,  1106,  1159,  1260,  1650,  2873,  2914,
        2938,  3232,  3359,  3588,  4373,  4414,  5311,  6015,  6272,
        6584,  6607,  8146,  9086,  9524,  9620, 10057, 12416, 12950,
       14154, 14172, 14190, 15330, 16691, 17191, 17768, 18644, 18841,
       19778, 20456, 22360, 22539, 22927, 23195, 24298, 24684, 26788,
       27578, 28047, 29040, 29288, 31047, 31107, 31228, 31862, 31863,
       32772, 33172, 33348, 33676, 33794, 33820, 33953, 35266, 37387,
       37784, 38588, 41284, 43375, 43483, 43993, 44232, 45398, 46297,
       46314])

In [134]:
# target_user_id = np.random.randint(0, 15)
target_user_id = 9995
with open(BOTIFY_DATA_DIR + "recommendations_als.json", "w") as rf:
    for target_user_id in range(10000):
        
        try:
            liked = train[target_user_id].nonzero()[1]
            recs = model.recommend(target_user_id, 
                                   csr_matrix(train[target_user_id]), 
                                   N=100, 
                                   filter_items=list(liked))
        except:
            continue
        
        if len(recs[0]) == 0:
            continue
            
        recommendation = {
            "user": int(target_user_id),
            "tracks": list(map(int, recs[0]))
        }
        rf.write(json.dumps(recommendation) + "\n")
    

In [115]:
recs

(array([ 1997,   156,  1539,   410,   616,  1486,  2793,  1497, 10185,
        20359,  1996,  1995,  1504,  1080,   246,  3690,  7918,  3916,
          175,  1525]),
 array([0.00395108, 0.0005525 , 0.00052259, 0.00040071, 0.00038647,
        0.00032606, 0.00032268, 0.00031142, 0.00030357, 0.00030274,
        0.00029908, 0.00028395, 0.00027435, 0.00027297, 0.00026902,
        0.00026319, 0.00025836, 0.00024016, 0.00023109, 0.00022303],
       dtype=float32))

In [135]:
tracks = pd.read_json("../tracks.json", lines=True)[["artist", "title", "track"]].copy()
tracks.head(3)

,artist,title,track
0,Jack Johnson,The Cove,0
1,Billy Preston,Nothing from Nothing,1
2,Paco De Lucia,Entre Dos Aguas,2


In [136]:
[tracks[tracks.track == cur_track].title.iloc[0] for cur_track in recs[0]]

['Cosmic Love',
 'Shape Shifter',
 'Lucky (Album Version)',
 'Drop The World',
 "If I Ain't Got You",
 'It Might Be You (from "Tootsie")',
 "It's New To Me",
 'The Saltwater Room',
 'Barcelona',
 "You're The One",
 'Revelry',
 'U Smile',
 'The Invisible Man',
 "Livin' On A Prayer",
 'Dog Days Are Over (Radio Edit)',
 'Summer',
 'Kryptonite',
 'Sous le soleil',
 "I Think I'll Live",
 'Gabriela Solo',
 'In My Place',
 'So In Love (The Real Deal) (Frankie Foncett Vocal Mix)',
 'Your Song (Alternate Take 10)',
 'Make Love To Your Mind',
 'Slip Away (Album)',
 'Full Circle',
 'Wait A Minute',
 'Teach Me How To Dougie',
 'The Gift',
 'Dental Care',
 'Mr. Me Too',
 'A Little More (Lay It Down Album Version)',
 'Rabbit Heart (Raise It Up)',
 'Nightrain',
 'Woman A Pressure Me',
 'Never Went To Church',
 'Alejandro',
 'Closer To You',
 "I CAN'T GET STARTED",
 'König der Löwen',
 'My Name Is',
 'Strings',
 'Back To Black',
 'River',
 'Nobody Not Really (Interlude)',
 'Drag You Down',
 'Rianna',
